### Header

In [21]:
# import libraries

# maths
import numpy as np
import pandas as pd
#import scipy.stats as stats
#from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# modelling
# from sklearn import linear_model
# from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
# from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
# from sklearn.preprocessing import StandardScaler,PolynomialFeatures
# from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
# from sklearn.pipeline import make_pipeline,Pipeline

# nlp
# from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from nltk.stem import PorterStemmer,WordNetLemmatizer
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import spacy
# from spacy.tokens import Doc

# web
#import requests
#import json

# others
import os
import re
import time
import datetime as datetime

In [29]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Import Data

In [30]:
train = pd.read_csv(input_path+'train.csv')
test = pd.read_csv(input_path+'test.csv')


### Inspect Data

In [24]:
train.head()


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [25]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


### Clean Data

In [32]:
def create_yr(x): 
    return x.split('-')[0] 

def create_mth(x): 
    return x.split('-')[1] 

def create_day(x): 
    return x.split('-')[2] 

def rename_columns (columns):
    return [column.lower() for column in columns]

def clean_data(df): 
    df['year'] = df.Date.apply(create_yr)
    df['month'] = df.Date.apply(create_mth)
    df['day'] = df.Date.apply(create_day)
    df['latitude'] = df.Latitude.apply(int)
    df['longitude'] = df.Longitude.apply(int)
    df.drop(['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'Date', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df.columns = rename_columns(df.columns)
    return df

train = clean_data(train)
test = clean_data(test)

### Output Data

In [33]:
train.head()

,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,1,0,2007,05,29,41,-87
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,1,0,2007,05,29,41,-87
2,CULEX RESTUANS,62,N MANDELL AVE,T007,1,0,2007,05,29,41,-87
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,1,0,2007,05,29,41,-87
4,CULEX RESTUANS,79,W FOSTER AVE,T015,4,0,2007,05,29,41,-87


In [34]:
test.head()

,id,species,block,street,trap,year,month,day,latitude,longitude
0,1,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
1,2,CULEX RESTUANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
2,3,CULEX PIPIENS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
3,4,CULEX SALINARIUS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
4,5,CULEX TERRITANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87


In [35]:
train.to_csv(output_path+'train.csv')
test.to_csv(output_path+'test.csv')